# Writing TTrees

As of now, uproot can write TTrees whose branches are basic types (integers and floating-point numbers).  

Basic usage - 

In [1]:
import uproot
import numpy

branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
with uproot.recreate("example.root") as f:
    f["t"] = tree
    f["t"].extend({"branch": numpy.array([1, 2, 3, 4, 5])})

Reading it back in uproot - 

In [2]:
f = uproot.open("example.root")
f["t"].array("branch")

array([1, 2, 3, 4, 5], dtype=int32)

Reading it back in ROOT -

In [3]:
import ROOT

f = ROOT.TFile.Open("example.root")
tree = f.Get("t")
treedata = tree.AsMatrix(["branch"])
print(treedata)

Welcome to JupyROOT 6.18/00
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]]


You can look at the current state of the TTree when you are putting in data -

In [4]:
import uproot

branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
f = uproot.recreate("example.root")
f["t"] = tree
f["t"]["branch"].newbasket([1, 2, 3, 4, 5])

Read it - 

In [5]:
f["t"].array("branch")

array([1, 2, 3, 4, 5], dtype=int32)

Keep writing - 

In [6]:
f["t"]["branch"].newbasket([6, 7, 8, 9, 10])

Close the file!

In [7]:
f.close()

Just to check if it is a valid ROOT file - 

In [8]:
import ROOT

f = ROOT.TFile.Open("example.root")
tree = f.Get("t")
treedata = tree.AsMatrix(["branch"])
print(treedata)

[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]


## Getting into the specifics

### Creating the Tree

This is how you create a TTree - 

In [9]:
t = uproot.newtree({"branch1": int, "branch2": numpy.int32, "branch3": uproot.newbranch(numpy.float64, title="This is the title")})

uproot.newtree() takes a python dictionary as an argument, where the key is the name of the branch and the value is the branch object or type of branch.

In [10]:
branchdict = {"branch2": ">i8"}

We can specify the title, the flushsize and the compression while creating the tree.  

This is an example of how you would add a title to your tree - 

In [11]:
tree = uproot.newtree(branchdict, title="TTree Title")

### Creating the Branches

There are several ways to specify a branch -

In [12]:
b = "int32"
# b = ">i4"

where we are using a string with the desired type

or

In [13]:
b = numpy.int32
# b = numpy.dtype(numpy.int32)

where we are using a numpy object or numpy dtype

or

In [14]:
b = uproot.newbranch("int32")
# b = uproot.newbranch(">i4")

The above method provides a lot more flexibility, letting you specify the flushsize, title of the branch and compression of the baskets along with the type of the branch.

To specify the title of the branch, similar to how you would add a title to a tree -

In [16]:
b = uproot.newbranch("int32", title="This is the title")

#### Writing the TTree to the file

In [17]:
f = uproot.recreate("demo.root") # Creates a writeable ROOT file
f["t"] = t # Writes the TTree to the file

### Writing baskets

We have 2 branches in our TTree -
* branch1
* branch2
* branch3

The suggested interface of writing baskets to the TTree is using the extend method -

In [18]:
f["t"].extend({"branch1": numpy.array([1, 2, 3, 4, 5]), "branch2": [6, 7, 8, 9, 10]})

The extend method takes a dictionary where the key is the name of the branch and the value of the dictionary is a numpy array or a list of data to be written to the branch.  
Remember to add equal number of entries to each branch, else you will run into an error.

In [19]:
f["t"].extend({"branch1": numpy.array([1, 2, 3, 4, 5]), "branch2": [6, 7, 8, 9]})

Exception: Baskets of all branches should have the same length


You can specify a flush parameter to True or False in the extend method.

In [20]:
f["t"].extend({"branch1": numpy.array([1, 2, 3, 4, 5]), "branch2": [6, 7, 8, 9, 10]}, flush=True)

By default, it is true. This means that these values are immediately flushed to the file. 

**Low level interface**  

If you want, you can write a basket to only 1 branch. But remember to add equal number of baskets to the other branches as well as ROOT assumes that all the branches have equal number of baskets and will not read the non-uniform baskets.

In [ ]:
f["t"]["branch1"].newbasket([1, 2, 3])

Add 3 more baskets to branch2!

In [ ]:
f["t"]["branch2"].newbasket([91, 92, 93])

## Compression

By default, the baskets of all the branches are compressed depending on the compression set for the file.

In [ ]:
branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch"].newbasket([1]*1000)

In the above example the baskets in the branch are compressed using LZMA with level equal to 5.

You can specify the compression of all the branches if you want it to be separate from the compression specified for the entire file.

In [ ]:
branchdict = {"branch": "int32", "testbranch": "int64"}
tree = uproot.newtree(branchdict, compression=uproot.LZ4(4)) # <-- LOOK HERE
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch"].newbasket([1]*1000)
    f["t"]["testbranch"].newbasket([2]*1000)

In the above example, the baskets in both branch "branch" and "testbranch" are compressed using LZ4 with level equal to 4.
If the user adds other objects(new tree, histogram, strings) to the ROOT file, they will still follow the compression specified in the file of LZMA with level equal to 5.

You can also specify the compression of each branch individually.

In the "Creating the Branches" section, we spoke about the compression option if one is using the newbranch interface to create the branch.

In [ ]:
b1 = uproot.newbranch("i4", compression=uproot.ZLIB(5))
b2 = uproot.newbranch("i8", compression=uproot.LZMA(4))
b3 = uproot.newbranch("f4")

branchdict = {"branch1": b1, "branch2": b2, "branch3": b3}
tree = uproot.newtree(branchdict, compression=uproot.LZ4(4))
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch1"].newbasket([1]*1000)
    f["t"]["branch2"].newbasket([2]*1000)
    f["t"]["branch3"].newbasket([3]*1000)

In the above example, the baskets in branch "branch1" are compressed using ZLIB with level equal to 5.  
The baskets in branch "branch2" are compressed using LZMA with level equal to 4.  
The baskets in branch "branch3" follow the compression set using the newtree interface and is using LZ4 with level equal to 4.  